In [1]:
import pandas as pd
import nltk, re
from nltk import word_tokenize, pos_tag
from sklearn.model_selection import train_test_split
import re, string
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import Binarizer, LabelBinarizer

In [2]:
df = pd.read_csv('article_summary-202001.csv') #reads the summary data
df2 = pd.read_csv('score_pred') #reads the data where we have manually rated the articles
df2 = df2[['title','sentences']]

In [3]:
# Joins both the dataframes
data = pd.merge(df,df2,left_on='Article',right_on='title')
data = data.rename(columns = {'Target Complexity': 'score'})

# Splits the string and converts it into a list
data['Article Topics Distribution'] = data['Article Topics Distributions'].apply(lambda x:x[1:-1].split(','))
data['Articles Vector Centroid'] = data['Article Vector Centroid'].apply(lambda x:x[1:-1].split())
data = data.drop(['Unnamed: 0','title','Article Vector Centroid', 'Article Topics Distributions'],axis=1)

In [4]:
# Splits the vectors into multiple columns
data[['ATD_1','ATD_2','ATD_3', 'ATD_4','ATD_5']] = pd.DataFrame(data['Article Topics Distribution'].values.tolist(), index= data.index)
new_data = pd.DataFrame(data['Articles Vector Centroid'].values.tolist(), index= data.index).add_prefix('AVC_')
data = pd.concat([data, new_data[:]], axis=1)
data = data.drop(['Article Topics Distribution','Articles Vector Centroid'],axis=1)

In [5]:
# filters the sentences to remove unneccesary characters and creates a description of the article
final_ls = []
overall_list = []
for i in range(0, len(data)):
    tokenize = ''
    result_list = []
    fil_sent = data['sentences'][i]
    line = re.sub('[!@#$]', '', fil_sent)
    tokenize= word_tokenize(line)
    for j in tokenize:
        result = j.split('"=')
        result = re.sub('[\W_]+', '', j)
        result_list.append(result)
        result_list = [x for x in result_list if x]
    final_ls.append(result_list)

joined_list = []
for i in range(0, len(final_ls)):
    res = ' '.join(final_ls[i])
    res = [res]
    joined_list.append(res)

In [6]:
data['content'] = joined_list
data['description'] = data.content.apply(' '.join)
data = data.drop(['sentences','content'],axis=1)
data = data[data.score != 0]
data['Eigenvector Centrality'].fillna(0, inplace=True)
data['Louvain Community'].fillna(0, inplace=True)

In [7]:
data.head(2)

Snapshot                                   Article  \
0    202001  Appellate procedure in the United States   
1    202001                        Politics of Angola   

   TextStat Fleisch Reading Difficulty  Eigenvector Centrality  \
0                             -1915.57                     0.0   
1                              -615.70                     0.0   

   Louvain Community  Clicks in month  Article Length  score  \
0                0.0              NaN           20512    2.0   
1                0.0              NaN            6997    1.0   

                ATD_1                 ATD_2  ...       AVC_91       AVC_92  \
0  0.7612398862838745   0.19284477829933167  ...  -0.09622715   0.35099052   
1   0.989683985710144                   nan  ...  -0.53916932  -0.00818249   

        AVC_93       AVC_94      AVC_95       AVC_96       AVC_97  \
0   -0.1076387  -0.24741805   0.0702673  -0.01548856  -0.53353546   
1  -0.05367916  -0.32687936  0.14850902   0.36836393  -0.65680124   

        AVC_98       AVC_99                                        description  
0  -0.74067314  -0.55266951  United States appellate procedure involves the...  
1  -0.79493512  -0.56443335  Since the adoption of a new the politics of An...  

[2 rows x 114 columns]

In [8]:
# Iterate through the columns to select the numeric one's and converts them to float
for col in list(data.columns):
    if ('Difficulty' in col or 'Centrality' in col or 'Community' in col or 
        'Clicks' in col or 'Length' in col or 'score' in col or 'ATD' in col or 'AVC' in col):
        data[col] = data[col].astype(float)

In [9]:
data.describe()

Snapshot  TextStat Fleisch Reading Difficulty  Eigenvector Centrality  \
count     205.0                           205.000000                   205.0   
mean   202001.0                          -854.965366                     0.0   
std         0.0                           988.799337                     0.0   
min    202001.0                         -9250.300000                     0.0   
25%    202001.0                         -1140.790000                     0.0   
50%    202001.0                          -579.500000                     0.0   
75%    202001.0                          -298.670000                     0.0   
max    202001.0                            67.760000                     0.0   

       Louvain Community  Clicks in month  Article Length       score  \
count              205.0        72.000000      205.000000  205.000000   
mean                 0.0     45048.819444     9405.673171    1.673171   
std                  0.0     72259.860261    10440.718352    0.737984   
min                  0.0        61.000000      108.000000    1.000000   
25%                  0.0      4239.000000     3493.000000    1.000000   
50%                  0.0     19031.500000     6506.000000    2.000000   
75%                  0.0     61242.750000    12412.000000    2.000000   
max                  0.0    436807.000000    97473.000000    3.000000   

            ATD_1       ATD_2      ATD_3  ...      AVC_90      AVC_91  \
count  144.000000  112.000000  63.000000  ...  205.000000  205.000000   
mean     0.445348    0.362726   0.133019  ...    0.533419   -0.169923   
std      0.350557    0.361920   0.198976  ...    0.168109    0.170271   
min      0.014058    0.010827   0.010547  ...   -0.011421   -0.789927   
25%      0.110318    0.034132   0.025818  ...    0.429883   -0.272177   
50%      0.337236    0.194741   0.059178  ...    0.542896   -0.172495   
75%      0.761425    0.723779   0.127499  ...    0.641947   -0.051073   
max      0.998625    0.998907   0.938126  ...    1.052888    0.244390   

           AVC_92      AVC_93      AVC_94      AVC_95      AVC_96      AVC_97  \
count  205.000000  205.000000  205.000000  205.000000  205.000000  205.000000   
mean     0.027212    0.013968   -0.082721    0.114379    0.054510   -0.751472   
std      0.179848    0.221194    0.189682    0.139415    0.228054    0.199080   
min     -0.588951   -0.968910   -0.599672   -0.304713   -0.493108   -1.325670   
25%     -0.071958   -0.123999   -0.209779    0.036160   -0.095248   -0.879760   
50%      0.036307    0.019076   -0.088677    0.123734    0.051077   -0.727168   
75%      0.138556    0.160862    0.059380    0.192328    0.216686   -0.598460   
max      0.578184    0.588779    0.410626    0.590148    0.675440   -0.393398   

           AVC_98      AVC_99  
count  205.000000  205.000000  
mean    -0.564090   -0.397062  
std      0.309672    0.149869  
min     -1.477485   -0.769623  
25%     -0.770936   -0.514428  
50%     -0.578957   -0.410901  
75%     -0.361412   -0.307139  
max      0.204514    0.101432  

[8 rows x 112 columns]

In [10]:
# # the plot show the distribution of the article complexity score
# plt.style.use('fivethirtyeight')
# plt.hist(data['score'].dropna(), bins = 100, edgecolor = 'k');
# plt.xlabel('Score'); plt.ylabel('Number of articles'); 
# plt.title('Article Score Distribution');

In [11]:
# # This plot shows the distribution of the length of the articles
# plt.figure(figsize=(8,8)) 
# plt.hist(data['Article Length'].dropna(), bins = 20, edgecolor = 'black');
# plt.xlabel('Article length'); 
# plt.ylabel('Count'); plt.title('Distribution of the article length');

In [12]:
# # Histogram Plot for the distributions of the number of clicks per articles
# plt.figure(figsize=(5,5)) 
# plt.hist(data['Clicks in month'].dropna(), bins = 20, edgecolor = 'black');
# plt.xlabel('Number of Clicks'); 
# plt.ylabel('Articles'); plt.title('Distribution of number of clicks');

In [13]:
# to check to see what impacts the score calculation
correlations_data = data.corr()['score'].sort_values()
correlations_data

AVC_31                   -0.472366
AVC_59                   -0.426826
AVC_32                   -0.401670
AVC_24                   -0.359475
AVC_88                   -0.358535
                            ...   
AVC_29                    0.522246
score                     1.000000
Snapshot                       NaN
Eigenvector Centrality         NaN
Louvain Community              NaN
Name: score, Length: 112, dtype: float64

In [14]:
'''this part of the code is to find the correlation between the features and the target'''
# Select the numeric columns
numeric_subset = data.select_dtypes('number')
numeric_subset.fillna(0, inplace=True) #replace nan values with 0

# Create columns with square root and log of numeric columns
for col in numeric_subset.columns:
    if col == 'score':
        next
    else:
        numeric_subset['sqrt_' + col] = np.sqrt(numeric_subset[col])
        numeric_subset['log_' + col] = np.log(numeric_subset[col])

# Select the categorical columns and one-hot encode them
categorical_subset = data[['Article', 'description']]
categorical_subset = pd.get_dummies(categorical_subset)

#combinig the numerical and categorical data
features = pd.concat([numeric_subset, categorical_subset], axis = 1)

# Find correlations with the score 
correlations = features.corr()['score'].dropna().sort_values()

/home/suraj/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs
/home/suraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/suraj/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/suraj/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: Runt

In [15]:
correlations.head()

sqrt_AVC_1    -1.000000
log_AVC_1     -1.000000
AVC_31        -0.472366
sqrt_AVC_78   -0.453497
log_AVC_78    -0.451312
Name: score, dtype: float64

In [16]:
print(numeric_subset.shape)
print(categorical_subset.shape)
print(features.shape)

(205, 334)
(205, 410)
(205, 744)


In [17]:
features = data.copy()

# Select the numeric columns
numeric_subset = data.select_dtypes('number')
numeric_subset.fillna(0, inplace=True)
# numeric_subset = numeric_subset.drop(['AVC_1'], axis=1)

# Select the categorical columns and one hot encode it
categorical_subset = data[['Article', 'description']]
categorical_subset = pd.get_dummies(categorical_subset) 

#combinig the numerical and categorical data
features = pd.concat([numeric_subset, categorical_subset], axis = 1)

print(numeric_subset.shape)
print(categorical_subset.shape)
print(features.shape)

(205, 112)
(205, 410)
(205, 522)


In [18]:
#making sure none of the score values are null
score = features[features['score'].notnull()] 

#seperating the column we intend to predict
features = score.drop(columns='score')
targets = pd.DataFrame(score['score']) 

# Split into 70% training and 30% testing set
X, X_test, y, y_test = train_test_split(features, targets, test_size = 0.3, shuffle=False)

print(X.shape)
print(X_test.shape)
print(y.shape)
print(y_test.shape)

(143, 521)
(62, 521)
(143, 1)
(62, 1)


In [19]:
'''Check the importance of features for prediction'''
# feat_model = LogisticRegression().fit(X,y)
# feature_importance = abs(feat_model.coef_[0])

# new_feat_model = RandomForestRegressor().fit(X,y)
# feature_imp = new_feat_model.feature_importances_

# column = features.columns
# imp_df = pd.DataFrame(column, columns=['column_name'])

# imp_df['logistic_feature_importance'] = feature_importance
# imp_df['randomForest_feature_importance'] = feature_imp
# # imp_df = imp_df.sort_values(by=['feature_importance'], ascending=True)
# # imp_df = imp_df[imp_df.feature_importance != 0]
# # col_list = imp_df['column_name'].values
# imp_df.to_csv('feature_importance_validation.csv')

'Check the importance of features for prediction'

In [20]:
transformer = Binarizer(threshold=2, copy=False)
y_test = transformer.transform(y_test)
y = transformer.transform(y)

In [21]:
def mean_absolute_error(y_true, y_pred):
    return np.mean(abs(y_true - y_pred))

def fit_and_evaluate(model):
    model.fit(X, y)
    model_pred = model.predict(X_test)
    model_mae = mean_absolute_error(y_test, model_pred)
    
    return model_mae # Returns the performance metric

In [22]:
baseline_guess = np.median(y)

print('The baseline guess is a score of %0.2f' % baseline_guess)
print("Baseline Performance on the test set: MAE = %0.4f" % mean_absolute_error(y_test, baseline_guess))

The baseline guess is a score of 0.00
Baseline Performance on the test set: MAE = 0.1935


In [23]:
#Create the scaler object with a range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)# Fit on the training data

# Transform both the training and testing data
X = scaler.transform(X)
X_test = scaler.transform(X_test)

# Convert y to one-dimensional array (vector)
y = np.array(y).reshape((-1, ))
y_test = np.array(y_test).reshape((-1, ))

In [24]:
lr = LinearRegression()
lr_mae = fit_and_evaluate(lr)
print('Linear Regression Performance on the test set: MAE = %0.4f' % lr_mae)

lg = LogisticRegression()
lg_mae = fit_and_evaluate(lr)
print('Logistic Regression Performance on the test set: MAE = %0.4f' % lg_mae)

svm = SVR(C = 1000, gamma = 0.1)
svm_mae = fit_and_evaluate(svm)
print('Support Vector Machine Regression Performance on the test set: MAE = %0.4f' % svm_mae)

gradient_boosted = GradientBoostingRegressor(random_state=60)
gb_mae = fit_and_evaluate(gradient_boosted)
print('Gradient Boosted Regression Performance on the test set: MAE = %0.4f' % gb_mae)

Linear Regression Performance on the test set: MAE = 0.2445
Logistic Regression Performance on the test set: MAE = 0.2445
Support Vector Machine Regression Performance on the test set: MAE = 0.2530
Gradient Boosted Regression Performance on the test set: MAE = 0.2473


In [25]:
# Create the model to use for hyperparameter tuning
model = LogisticRegression().fit(X, y)

final_pred_prob = model.predict_proba(X_test)
final_pred = model.predict(X_test)

print('Final model performance on the test set:   MAE = %0.4f.' % mean_absolute_error(y_test, final_pred))

Final model performance on the test set:   MAE = 0.1613.


/home/suraj/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [26]:
actual, predicted = train_test_split(data, test_size = 0.3, shuffle=False)

In [27]:
predicted['binary_score'] = y_test
predicted['difficulty'] = final_pred.tolist()
predicted['probability'] = final_pred_prob.tolist()
predicted = predicted[['Article','description','score','binary_score','difficulty','probability']] 

/home/suraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/suraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/suraj/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [28]:
dict_values = {1.0:'Hard', 0.0:'Easy'}
predicted = predicted.replace({"difficulty": dict_values})
predicted['no_of_words'] = predicted.description.apply(lambda x: len(str(x).split(' ')))
predicted.head()

Article  \
143      Montevideo Convention   
144  ML (programming language)   
145                Monoid ring   
146              North America   
147                 Nominalism   

                                           description  score  binary_score  \
143  type datedrafted datesigned December 26 1933 l...    2.0           0.0   
144  designer and others at the developer latest re...    2.0           0.0   
145  In a monoid ring is a constructed from a ring ...    2.0           0.0   
146  nbsp population 579024000 2016 density 229km s...    1.0           0.0   
147  Nominalism is a view which comes at least in t...    3.0           1.0   

    difficulty                                 probability  no_of_words  
143       Easy  [0.9616880495550858, 0.038311950444914214]          331  
144       Easy    [0.6342788758593384, 0.3657211241406616]         1241  
145       Hard    [0.3762900014611731, 0.6237099985388269]          436  
146       Easy   [0.971377822945586, 0.028622177054414104]         1933  
147       Easy    [0.9586438989558362, 0.0413561010441638]          565

In [29]:
predicted = predicted[['Article', 'no_of_words', 'difficulty']]
predicted

Article  no_of_words difficulty
143      Montevideo Convention          331       Easy
144  ML (programming language)         1241       Easy
145                Monoid ring          436       Hard
146              North America         1933       Easy
147                 Nominalism          565       Easy
..                         ...          ...        ...
200                       Mort         1184       Easy
201            Tim Berners-Lee          728       Easy
202               The Big Test           18       Easy
203                  Tradition         1678       Easy
204              Thump Records          106       Easy

[62 rows x 3 columns]

In [30]:
# predicted['read_time'] = predicted['no_of_words'] / 250
# predicted = predicted[['Article', 'no_of_words', 'difficulty','read_time']]
# predicted